In [ ]:
import pandas as pd


import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import math
import time
import h5py
import sklearn
from sklearn.utils import shuffle

import os

print('Tensorflow Version {}'.format(tf.__version__))

In [ ]:
start_notebook=time.time()

## Hyperparameters

In [ ]:
class HParams(object):
    def __init__(self):
        self.Lambda = 1
        self.batchsize = 50
        self.trainingBatchSize = 5

HPARAMS = HParams()

## Functions

In [ ]:
def printTime(delta):
    
    h = (delta // 3600)
    m = (delta % 3600) // 60
    s = delta % 60
    
    if (delta > 3600):
        print('Time {0:.0f}h {1:.0f}m {2:.2f}s'.format(h, m, s))
        
    elif (delta > 60):
        print('Time {0:.0f}m {1:.2f}s'.format(m, s))
    
    else:
        print('Time {0:.2f}s'.format(s))

In [ ]:
alpha = 0.5

def plotFeature(X, Y, feat):
    
    siglab = Y[:,1]==1
    bkglab = Y[:,1]==0
    
    
    
    sig = X[feat][siglab].flatten()
    bkg = X[feat][bkglab].flatten()
    
    maximum = np.max([np.max(sig), np.max(bkg)])
    minimum = np.min([np.min(sig), np.min(bkg)])
    nbins = 50
    brange = np.linspace(minimum, maximum, 50)
    
    
    plt.hist(sig, bins=brange, hatch='//', alpha=alpha, label='Signal')
    plt.hist(sig, bins=brange, histtype='step', color='k')
    plt.hist(bkg, bins=brange, hatch='\\', alpha=alpha, label='Background')
    plt.hist(bkg, bins=brange, histtype='step', color='k')
    
    plt.xlabel(feat)
    plt.ylabel('Counts')
    
    plt.legend()
    plt.show()
    



In [ ]:
def getAdversary(X, X_shift, Y):
    
    sig = (Y[:,1]==1)
    
    xval = X[sig].flatten()
    xval_s = X_shift[sig].flatten()

    maximum = np.max([np.max(X), np.max(X_shift)])
    minimum = np.min([np.min(X), np.min(X_shift)])
    nbins = 50
    brange = np.linspace(minimum, maximum, 50)

    hx = plt.hist(xval, bins=brange, histtype='step', color='k', label='Signal')
    hxs = plt.hist(xval_s, bins=brange, histtype='step', color='darkgray', label='Perturbed Signal')
    plt.legend()
    plt.show()
    
    hx_height = hx[0]
    hxs_height = hxs[0]
    height = hx_height-hxs_height
    xaxis = hx[1][1:]
   
    plt.plot(xaxis, height, color='k', linewidth=2, label='Adversary')
    plt.legend()
    
    label_adv = np.ones_like(height)

    plt.show()
    
    
    return height, label_adv
    


In [ ]:
if not os.path.isdir("model"):
    os.mkdir("model")

## Get datasets as dictionaries
### Datasets generated with ToyModel/makeFourVectors.ipynb

In [ ]:
# created by running makeFourVectors.ipynb with addPerturbation = False
data_train = np.load('data/jetConstTrain_overlap.npz')
data_test = np.load('data/jetConstTest_overlap.npz')

# created by running makeFourVectors.ipynb with addPerturbation = True
data_train_shift = np.load('data/jetConstTrain_overlap_perturb.npz')
data_test_shift = np.load('data/jetConstTest_overlap_perturb.npz')


In [ ]:
feat_all = ['pt', 'eta', 'phi', 'mass', 'thetalab', 'radiilab', 'zlab']


feat_list = ['thetalab', 'pt', 'eta', 'phi']
feat_xaug = []

Nlist = len(feat_list)
Nxaug = len(feat_xaug)

nEvents = len(data_train['pt'].flatten())

print('Number of particle list features: ', Nlist)
print('Number of XAUG features: ', Nxaug)
print('N Events: ', nEvents)

In [ ]:
# get datasets as dictionaries

X_train = {key: data_train[key] for key in feat_list}
X_test = {key: data_test[key] for key in feat_list}

X_train_shift = {key: data_train_shift[key] for key in feat_list}
X_test_shift = {key: data_test_shift[key] for key in feat_list}



# get labels

Y_train = data_train['labels']
Y_test = data_test['labels']

Y_train_shift = data_train_shift['labels']
Y_test_shift = data_test_shift['labels']

In [ ]:
for key, var in data_test.items():
    print(var.shape, key)

In [ ]:
layer_names = [key for key in X_train.keys() if not 'labels' in key]
layer_names

## Make $\mathrm{log}(p_{T})$ and $\mathrm{log}(z)$

In [ ]:
X_train['pt'] = np.log(X_train['pt'])
# X_train['zlab'] = np.log(X_train['zlab'])



In [ ]:
plotFeature(X_train, Y_train, 'thetalab')

In [ ]:
plotFeature(X_train, Y_train, 'pt')

In [ ]:
plotFeature(X_train, Y_train, 'eta')

In [ ]:
plotFeature(X_train, Y_train, 'phi')

## Signal difference adversary
Note: This is not currently used as an input

In [ ]:
plotFeature(X_train, Y_train, 'thetalab')

In [ ]:
plotFeature(X_train_shift, Y_train_shift, 'thetalab')

In [ ]:
X_adv, Y_adv = getAdversary(X_train['thetalab'], X_train_shift['thetalab'], Y_train)

## Bin $\Delta \theta$
`binned_theta` is an input to the adversary model and
`theta_labels` are the labels for the adversary model

In [ ]:
# reshape theta
X_train['thetalab'] = X_train['thetalab'][:,0]

thetabins = np.linspace(0, 0.3, 11)
binned_theta = np.digitize(X_train['thetalab'], thetabins) + 1
theta_labels = tf.one_hot(binned_theta.squeeze(), 11)

In [ ]:
print('\nfirst 5 bins\n')
print(binned_theta[:5])
print('\nfirst 5 labels\n')
print(theta_labels.numpy()[:5])

In [ ]:
plt.hist(X_train['thetalab'].flatten(), bins=thetabins, histtype='step', density=True)
plt.xlabel(r'$\Delta \theta$')
plt.ylabel('Density')
plt.title(r'Binned $\Delta \theta$')
plt.show()

## Classifier

In [ ]:
def build_classifier(n, nex):
    inpts = []
    xlayers = []
    
    for i in range(n):
        
        if('theta' in layer_names[i]):


            inpt = tf.keras.Input(shape = (1,), name=layer_names[i])

            x = tf.keras.layers.Flatten()(inpt)
            inpts.append(inpt)
            xlayers.append(x)

            x_dense = tf.keras.layers.Dense(64, activation='relu')(x)
            xlayers.append(x_dense)
            
        else:
            
            inpt = tf.keras.Input(shape = (10,1), name=layer_names[i])
            x = tf.keras.layers.Flatten()(inpt)

            inpts.append(inpt)
            xlayers.append(x) 
    
    if(n > 1):
        x = tf.keras.layers.concatenate(inputs=xlayers, axis=-1)
        
    if(n==1):
        Inputs = inpts[0]
    else:
        Inputs = inpts
        
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    
    
    output = tf.keras.layers.Dense(2, activation='softmax', name = 'labels')(x) 
    model = tf.keras.Model(inputs=Inputs, outputs=output, name='Classifier')
    
    
    return model
    

In [ ]:
model_class = build_classifier(Nlist, Nxaug)
model_class.summary()

In [ ]:
# def build_classifier(n, nex):
    
#     inpts = []
#     xaugs = []
#     xlayers = []
    

    
#     # loop over all input variables
#     for i in range(n):
        
#         # particle list inputs 
#         if(i < n-nex):
            
#             if('theta' in layer_names[i]):
#                 inpt = tf.keras.Input(shape = (1,), name=layer_names[i])
                
#                 x = tf.keras.layers.Flatten()(inpt)

#                 inpts.append(inpt)
#                 xlayers.append(x)
                
#                 x_dense = tf.keras.layers.Dense(64, activation='relu')(x)
#                 xlayers.append(x_dense)
                
#             else:
#                 inpt = tf.keras.Input(shape = (10,1), name=layer_names[i])



# #             x = tf.keras.layers.Conv1D(64, 3, padding = 'same', activation='relu')(inpt)
# #             x = tf.keras.layers.Conv1D(64, 1, padding = 'same', activation='relu')(x)
# #             x = tf.keras.layers.Dropout(0.2)(x)
# #             x = tf.keras.layers.MaxPool1D(2)(x)
# #             x = tf.keras.layers.Conv1D(32, 3, padding = 'same', activation='relu')(x)
# #             x = tf.keras.layers.Conv1D(32, 1, padding = 'same', activation='relu')(x)
# #             x = tf.keras.layers.Dropout(0.2)(x)
# #             x = tf.keras.layers.MaxPool1D()(x)

#                 x = tf.keras.layers.Flatten()(inpt)

#                 inpts.append(inpt)
#                 xlayers.append(x)
        
#         elif((nex > 0)):
#             inpt = tf.keras.layers.Input(shape = (1,), name=layer_names[i])
#             xaugs.append(inpt)
    
#     #concatenation of particle list inputs with expert variable inputs
    
#     if(n > 1):
#         x = tf.keras.layers.concatenate(inputs=xlayers+xaugs, axis=-1)
        
        
#     if(n==1):
#         Inputs = inpts[0]
#     else:
#         Inputs = inpts+xaugs
    
    
#     x = tf.keras.layers.Dense(128, activation='relu')(x)
#     x = tf.keras.layers.Dense(64, activation='relu')(x)
   
    
#     output = tf.keras.layers.Dense(2, activation='softmax', name = 'labels')(x) 
#     model = tf.keras.Model(inputs=Inputs, outputs=output, name='Classifier')
    
    
#     return model

In [ ]:

# model_class = build_classifier(Nlist, Nxaug)

In [ ]:
# model_class.summary()

## Adversary

In [ ]:
def build_adversary():
    
#     inputs = []
    
    input_from_model = tf.keras.layers.Input(shape = (2,), name='Input')
    x_input = tf.keras.layers.Flatten()(input_from_model)
        
    feature_input = tf.keras.layers.Input(shape = (1,), name='theta')
    x_feature = tf.keras.layers.Flatten()(feature_input)
    
#     x = tf.keras.layers.concatenate(inputs=[x_feature,x_input], axis=-1)
#     x = tf.keras.layers.Flatten()(x)

    x = tf.keras.layers.Dense(64, activation='relu', name='Adversary_Dense64_2')(x_feature)
    x = tf.keras.layers.Dense(128, activation='relu', name='Adversary_Dense128')(x)


    
    inputs = [input_from_model, feature_input]
    output = tf.keras.layers.Dense(11, activation='softmax', name = 'output')(x) 
        
    model = tf.keras.Model(inputs=inputs, outputs=output, name='Adversary')

    return model

In [ ]:
model2 = build_adversary()

In [ ]:
model2.summary()

## Classifier and Adversary Training Loop



### Initialize Datasets

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
train_dataset = train_dataset.batch(HPARAMS.batchsize)


In [ ]:
train_dataset_adv = tf.data.Dataset.from_tensor_slices((X_train['thetalab'], theta_labels))
train_dataset_adv = train_dataset_adv.batch(HPARAMS.batchsize)

In [ ]:
loss_class_fn = tf.keras.losses.CategoricalCrossentropy()
loss_adv_fn = tf.keras.losses.CategoricalCrossentropy()


optimizer_class=tf.keras.optimizers.Adam(lr=1e-3)
optimizer_adv=tf.keras.optimizers.Adam(lr=1e-3)

In [ ]:
model_class = build_classifier(Nlist, Nxaug)
model_adv = build_adversary()

## Custom Gradient

### Combined loss of classifier and adversary

$$ \mathcal{L}_{total} = \mathcal{L}_{classifier} - \lambda \mathcal{L}_{adversary} $$ 

In [ ]:
# default custom gradient from tensorflow
# change this

print('lambda = {}'.format(HPARAMS.Lambda))

@tf.custom_gradient
def calc_grad_total(x, y, x_a, y_a):
    
    # x   - predictions of classifier
    # y   - labels of classifier
    # x_a - predicitions of adversary
    # y_a - labels of adversary
    
    def grad(upstream):
        
        x0 = tf.where(tf.equal(x, 0), 1e-14, x)
        x0_a = tf.where(tf.equal(x_a, 0), 1e-14, x_a)

        one_minus_x = tf.where(tf.equal(x, 1), 1e-14, 1-x)
        one_minus_x_a = tf.where(tf.equal(x_a, 1), 1e-14, 1-x_a)
        
     
        dz_dy_c = -np.log(x0) + np.log(one_minus_x)

        dz_dx_c = (-y / x0) + (1 - y) / (one_minus_x)

        dz_dy_a = -np.log(x0_a) + np.log(one_minus_x_a) * (-HPARAMS.Lambda)
 
        dz_dx_a = (-y_a / x0_a) + (1 - y_a) / (one_minus_x_a) * (-HPARAMS.Lambda)
        
        
        return upstream * dz_dx_c, upstream * dz_dy_c, upstream * dz_dx_a, upstream * dz_dy_a
    
  
        
        
    # categorical cross entropy
    
    

    x0 = tf.where(tf.equal(x, 0), 1e-14, x)
    x0_a = tf.where(tf.equal(x_a, 0), 1e-14, x_a)
    
    x1 = tf.where(tf.equal(x, 1), 1e-14, 1-x)
    x1_a = tf.where(tf.equal(x_a, 1), 1e-14, 1-x_a)
    
    
    y = tf.cast(y, 'float32')
    
    
    z_c = -y * np.log(x0) - (1-y)*np.log(x1)   

    z_a = -y_a * np.log(x0_a) - (1-y_a)*np.log(x1_a)


    z = tf.math.reduce_mean(z_c) - HPARAMS.Lambda * tf.math.reduce_mean(z_a)
    
    return z, grad

    

## Train classifier and adversary separately for one epoch

In [ ]:
model_class  = build_classifier(Nlist, Nxaug)
model_adv = build_adversary()


logits_adv = np.zeros((HPARAMS.batchsize, 11))
y_batch_train_adv = np.zeros((HPARAMS.batchsize, 11))
classifier_output = np.zeros((HPARAMS.batchsize, 2))

begin = time.time()
last_epoch=time.time()
# train classifier first 
for epoch in range(1):
    
    print("\nClassifier update %d" % (1+epoch,))
    printTime(time.time() - last_epoch)
    last_epoch=time.time()

    # Iterate over the batches of the dataset.      
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):

        ### TRAIN CLASSIFIER ###
    
        with tf.GradientTape() as tape:
               
            # logits for the batch
            logits_class = model_class(x_batch_train)

            # calculate loss
            loss_class = loss_class_fn(y_batch_train, logits_class)
            
            # get and apply gradients
            grads_class = tape.gradient(loss_class, model_class.trainable_weights)
            
            
            optimizer_class.apply_gradients(zip(grads_class, model_class.trainable_weights))
            
    classifier_output = model_class(x_batch_train)
            

    
# train adversary first

last_epoch=time.time()
for epoch in range(3):
    
    print("\nAdversary update %d" % (1+epoch,))
    printTime(time.time() - last_epoch)
    last_epoch=time.time()

    # Iterate over the batches of the dataset.      
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset_adv):

        ### TRAIN ADVERSARY ###
    
        with tf.GradientTape() as tape:
               
            # logits for the batch
            logits_adv = model_adv([classifier_output, x_batch_train])

            # calculate loss
            loss_adv = loss_adv_fn(y_batch_train, logits_adv)
            
            
            adversary_weights = model_adv.trainable_weights
            
            
            # set adversary weights to 0
            
            #adversary_weights[0] = tf.Variable(adversary_weights[0].numpy()*[np.ones(64), np.zeros(64)], 
            #            name=adversary_weights[0].name, shape=adversary_weights[0].shape)
            
            
            # get and apply gradients
            grads_adv = tape.gradient(loss_adv, model_adv.trainable_weights)
            
            
            
            optimizer_adv.apply_gradients(zip(grads_adv, model_adv.trainable_weights))

            
            
            
print()            
print('L_class', loss_class.numpy())
print('L_adv', loss_adv.numpy())

## Training Loop

### Update each batch instead of each epoch

1. Train classifier for 10 batches (HPARAMS.trainingBatchSize) with $ \mathcal{L}_{total} = \mathcal{L}_{classifier} - \lambda \mathcal{L}_{adversary} $
2. Train adversary for 10 batches (HPARAMS.trainingBatchSize)
3. Repeat


In [ ]:
nBatches = nEvents // HPARAMS.batchsize
nBatchLoops = nBatches // HPARAMS.trainingBatchSize // 2
nBatchLoops


In [ ]:
start_training = time.time()

total_epochs=1



# inititalize loss weights
weights = []
weights_adv = []


nEpochs = 5
nSteps = nBatchLoops*nEpochs
# nSteps = nEpochs // HPARAMS.trainingBatchSize

print('Training for {0} epochs'.format(nEpochs))
toPrint=True

# train classifier
# skipped = 0
last_epoch=time.time()
for epoch in range(nEpochs):

    
    if(toPrint):
        print("\nEpoch %d" % (total_epochs,))
        printTime(time.time() - last_epoch)
        print('L_class', loss_class.numpy())
        print('L_adv', loss_adv.numpy())
        last_epoch = time.time()

        total_epochs += 1
#         toPrint=False

    
    
    
    # Iterate over the batches of the dataset.
    
#     if(skipped > nBatchLoops-10): 
#         skipped = 0
#         toPrint=True
        
    
#     clipped_dataset = train_dataset.skip(skipped).take(HPARAMS.trainingBatchSize)
#     clipped_dataset_adv = train_dataset_adv.skip(skipped).take(HPARAMS.trainingBatchSize)
    
    
#     for i, (x, y) in enumerate(clipped_dataset):
#         if(i==0): print(x['thetalab'][0].numpy())
    
#     skipped += HPARAMS.trainingBatchSize
    
    
    # Iterate over the batches of the dataset.      
    for step, (x_batch_train_adv, y_batch_train_adv) in enumerate(train_dataset_adv):

        ### TRAIN ADVERSARY ###
        

        with tf.GradientTape() as tape:

            # logits for the batch
            logits_adv = model_adv([classifier_output, x_batch_train_adv])

            # calculate loss
            loss_adv = loss_adv_fn(y_batch_train_adv, logits_adv)
            
            
            adversary_weights = model_adv.trainable_weights
            
            # set adversary weights to 0
            
            #adversary_weights[0] = tf.Variable(adversary_weights[0].numpy()*[np.ones(64), np.zeros(64)], 
            #            name=adversary_weights[0].name, shape=adversary_weights[0].shape)
            

            # get and apply gradients
            grads_adv = tape.gradient(loss_adv, model_adv.trainable_weights)
            optimizer_adv.apply_gradients(zip(grads_adv, model_adv.trainable_weights))      



    
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):


        ### TRAIN CLASSIFIER ###

        # logits for the batch
        logits_class = model_class(x_batch_train) 

        # calculate loss
        loss_class = loss_class_fn(y_batch_train, logits_class)
        
        with tf.GradientTape() as tape_adv:
            
            adversary_weights = model_adv.trainable_weights
            
            # set adversary weights to 0
            
            #adversary_weights[0] = tf.Variable(adversary_weights[0].numpy()*[np.ones(64), np.zeros(64)], 
            #            name=adversary_weights[0].name, shape=adversary_weights[0].shape)
            

            # logits for the batch
            logits_adv = model_adv([classifier_output, x_batch_train_adv])
#             logits_adv = model_adv([classifier_output, x_batch_train_adv])



            # calculate loss
            loss_adv = loss_adv_fn(y_batch_train_adv, logits_adv)
            


        with tf.GradientTape(persistent=True) as tape_class:

            # logits for the batch
            logits_class = model_class(x_batch_train) 

            
            # get loss of classifier
            loss_class = loss_class_fn(y_batch_train, logits_class)
            
            # get loss of adversary
            loss_adv = loss_adv_fn(y_batch_train_adv, logits_adv)
            
     
            
            
            tape_class.watch(logits_class)
            tape_class.watch(y_batch_train)
            tape_class.watch(logits_adv)
            tape_class.watch(y_batch_train_adv)
            
            
            
            # calculate total loss
#             grads_total = calc_grad_total(model_class.trainable_weights, model_adv.trainable_weights)

            
            grads_total = calc_grad_total(logits_class, y_batch_train, logits_adv, y_batch_train_adv)
            
           
            
            
            # get gradients from classifier
            grads_class_total = tape_class.gradient(grads_total, model_class.trainable_weights)            

            
            # apply gradients
            optimizer_class.apply_gradients(zip(grads_class_total, model_class.trainable_weights))
            
            
            
            
            weights = grads_class
            weights_adv = grads_adv
            
            
            classifier_output = model_class(x_batch_train)
            
            
            
            
            


#         classifier_output = model_class(x_batch_train)


end_training = time.time()
print('Total time: ', end='')
printTime(end_training - start_training)

In [ ]:


# model_adv = build_adversary()
# model_adv.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-3), 
#                   loss=tf.keras.losses.CategoricalCrossentropy()
#                  )



In [ ]:

# model_adv.fit(X_train['thetalab'], theta_labels, epochs=50)

## Plots

In [ ]:
xlabels = {
'theta':r'$\Delta \theta$',
'pt':r'$\mathrm{log}(p_{T})$',
'eta':r'$\eta$',
'phi':r'$\phi$',
'mass':r'Mass',  
}


In [ ]:
theta_avg = X_train['thetalab'].squeeze()
pt_avg = np.average(X_train['pt'].squeeze(), axis=1)
eta_avg = np.average(X_train['eta'].squeeze(), axis=1)
phi_avg = np.average(X_train['phi'].squeeze(), axis=1)


In [ ]:
bins2d = [np.linspace(0,0.4,11),np.linspace(0,1,11)]
vmin = 1e-1
vmax=0.5*1e4
norm=None

In [ ]:
# plt.scatter(theta_avg, predict_t2[:,1])
# plt.xlabel(r'$\Delta \theta$')
# # plt.xticks([2,3,4,5,6])
# plt.ylabel(r'Prediction')
# plt.show()

In [ ]:
# plt.scatter(np.average(X['pt'].squeeze(), axis=1), predict_t2[:,1])
# plt.xlabel(r'$p_{T}$')
# # plt.xticks([2,3,4,5,6])
# plt.ylabel(r'Prediction')
# plt.show()

In [ ]:

predict = model_class.predict(X_train)
predict_adv = model_adv.predict([predict, X_train['thetalab']])

In [ ]:
fig, ax = plt.subplots(2,3, figsize=(15,7.5))


ax[0][0].scatter(theta_avg, predict[:,1])
ax[0][0].set_xlabel(r'$\Delta \theta$')
ax[0][0].set_ylabel(r'Prediction')

ax[0][1].scatter(pt_avg.flatten(), predict[:,1])
ax[0][1].set_xlabel(xlabels['pt'])
ax[0][1].set_ylabel(r'Prediction')

ax[0][2].scatter(eta_avg, predict[:,1])
ax[0][2].set_xlabel(xlabels['eta'])
ax[0][2].set_ylabel(r'Prediction')

ax[1][0].scatter(phi_avg, predict[:,1])
ax[1][0].set_xlabel(xlabels['phi'])
ax[1][0].set_ylabel(r'Prediction')

plt.tight_layout()

figname = 'Lambda'+str(HPARAMS.Lambda)+'_scatterplots.png'
plt.savefig(figname)
print('saving '+figname)

plt.show()

In [ ]:
theta_labels[:,2]==1

In [ ]:
fig, ax = plt.subplots(4,3, figsize=(15,15))

ax[0][0].scatter(theta_avg, predict_adv[:,2])
ax[0][0].set_xlabel(r'$\Delta \theta$')
ax[0][0].set_ylabel(r'Prediction')
ax[0][0].set_title('Bin 3')
ax[0][0].set_ylim([0,1])

ax[0][1].scatter(theta_avg, predict_adv[:,3])
ax[0][1].set_xlabel(r'$\Delta \theta$')
ax[0][1].set_ylabel(r'Prediction')
ax[0][1].set_title('Bin 4')
ax[0][1].set_ylim([0,1])

ax[0][2].scatter(theta_avg, predict_adv[:,4])
ax[0][2].set_xlabel(r'$\Delta \theta$')
ax[0][2].set_ylabel(r'Prediction')
ax[0][2].set_title('Bin 5')
ax[0][2].set_ylim([0,1])

ax[1][0].scatter(theta_avg, predict_adv[:,5])
ax[1][0].set_xlabel(r'$\Delta \theta$')
ax[1][0].set_ylabel(r'Prediction')
ax[1][0].set_title('Bin 6')
ax[1][0].set_ylim([0,1])

ax[1][1].scatter(theta_avg, predict_adv[:,6])
ax[1][1].set_xlabel(r'$\Delta \theta$')
ax[1][1].set_ylabel(r'Prediction')
ax[1][1].set_title('Bin 7')
ax[1][1].set_ylim([0,1])

ax[1][2].scatter(theta_avg, predict_adv[:,7])
ax[1][2].set_xlabel(r'$\Delta \theta$')
ax[1][2].set_ylabel(r'Prediction')
ax[1][2].set_title('Bin 8')
ax[1][2].set_ylim([0,1])

ax[2][0].scatter(theta_avg, predict_adv[:,8])
ax[2][0].set_xlabel(r'$\Delta \theta$')
ax[2][0].set_ylabel(r'Prediction')
ax[2][0].set_title('Bin 9')
ax[2][0].set_ylim([0,1])

ax[2][1].scatter(theta_avg, predict_adv[:,9])
ax[2][1].set_xlabel(r'$\Delta \theta$')
ax[2][1].set_ylabel(r'Prediction')
ax[2][1].set_title('Bin 10')
ax[2][1].set_ylim([0,1])

ax[2][2].scatter(theta_avg, predict_adv[:,10])
ax[2][2].set_xlabel(r'$\Delta \theta$')
ax[2][2].set_ylabel(r'Prediction')
ax[2][2].set_title('Bin 11')
ax[2][2].set_ylim([0,1])

ax[3][0].scatter(theta_avg, predict_adv[:,0])
ax[3][0].set_xlabel(r'$\Delta \theta$')
ax[3][0].set_title('Bin 1')
ax[3][0].set_ylabel(r'Prediction')

ax[3][1].scatter(theta_avg, predict_adv[:,1])
ax[3][1].set_xlabel(r'$\Delta \theta$')
ax[3][1].set_title('Bin 2')
ax[3][1].set_ylabel(r'Prediction')


plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8, 4))
hist2d = plt.hist2d(theta_avg.flatten(), predict[:,1], norm=LogNorm())
plt.colorbar()
plt.tight_layout()
plt.xlabel(r'$\Delta \theta$')
# plt.xticks([2,3,4,5,6])
plt.ylabel(r'Prediction')
# plt.title('Second Training')
plt.tight_layout()
figname = 'Lambda'+str(HPARAMS.Lambda)+'_2D_theta.png'
plt.savefig(figname)
print('saving '+figname)
plt.show()

vmin = np.min(hist2d[0].flatten()[np.abs(hist2d[0].flatten()) > 0])
vmax = np.max(hist2d[0].flatten())

In [ ]:
plt.figure(figsize=(8, 4))
plt.hist2d(pt_avg.flatten(), predict[:,1], norm=LogNorm(), vmin=vmin, vmax=vmax)
plt.colorbar()
plt.tight_layout()
plt.xlabel(r'$p_{T}$')
# plt.xticks([2,3,4,5,6])
plt.ylabel(r'Prediction')
# pl t.title('Second Training')
plt.tight_layout()
figname = 'Lambda'+str(HPARAMS.Lambda)+'_2D_pt.png'
plt.savefig(figname)
print('saving '+figname)
plt.show()

mark down code for image

<!-- ![](factor100_2.png) -->



In [ ]:
printTime(time.time()-start_notebook)